<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       PolynomialFeaturesFit and PolynomialFeaturesTransform function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'>PolynomialFeaturesFit function stores all the specified values in the argument in a DataFrame format.
    All polynomial combinations of the features with degrees less than or equal to the specified degree are
    generated. For example, for a two-dimensional input sample [x, y], the degree-2 polynomial features are
    [x, y, x-squared, xy, y-squared, 1].<br>
    PolynomialFeaturesTransform function generates a feature matrix of all polynomial
    combinations of the feature by extracting the target column, degree, bias and interaction
    information from the output of the PolynomialFeaturesFit function.
<br> In this notebook we will see how we can use the PolynomialFeaturesFit and PolynomialFeaturesTransform functions available in Vantage to find non-linear relationships between different customer attributes.</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

In [ ]:
from teradataml import *

# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_PolynomialFeaturesFitandTransform_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_BankChurn_cloud');"        # Takes 30 seconds
#%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_BankChurn_local');" 

In [ ]:
%run -i ../../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
tdf = DataFrame(in_schema("DEMO_BankChurn", "customer_churn"))
print("Shape of the data: ", tdf.shape)
tdf

In [ ]:
tdf.tdtypes

In [ ]:
# Select relevant numerical columns for polynomial feature generation
target_columns = ["CreditScore", "Age", "Balance"]

<p style = 'font-size:16px;font-family:Arial'>A bank wants to analyze how customer demographics and financial behavior impact churn. Non-linear relationships might exist between customer attributes (like Age, CreditScore, and Balance) and churn. To capture these relationships, we generate polynomial features of selected numerical variables and use them in a predictive model.
</p>

In [ ]:
help(PolynomialFeaturesFit)

In [ ]:
# Apply PolynomialFeaturesFit to create polynomial terms up to degree 2
fit_result = PolynomialFeaturesFit(
    data=tdf,
    target_columns=target_columns,
    degree=2
)

<p style = 'font-size:16px;font-family:Arial'>Apply PolynomialFeaturesTransform to make changes in the dataset
</p>

In [ ]:
help(PolynomialFeaturesTransform)

In [ ]:
# Applying PolynomialFeaturesTransform
transformed_data = PolynomialFeaturesTransform(
    data=tdf,
    object=fit_result
)

<p style = 'font-size:16px;font-family:Arial'>The final output consists of an expanded feature set with polynomial transformations that capture non-linear relationships for columns CreditScore, Age and Balance enhancing predictive modeling for customer churn analysis.
</p>

In [ ]:
# Print the transformed DataFrame
transformed_data.result

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>3. Cleanup</b>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call remove_data('DEMO_BankChurn');"        # Takes 10 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;">
<b style = 'font-size:18px;font-family:Arial'>Dataset:</b>

- `RowNumber`: Row index
- `CustomerId`: Unique customer ID
- `Surname`: Customer's surname
- `CreditScore`: Credit score of the customer
- `Geography`: Country (Germany / France / Spain)
- `Gender`: Gender (Male / Female)
- `Age`: Age of the customer
- `Tenure`: Number of years the customer has been associated with the bank
- `Balance`: Account balance
- `NumOfProducts`: Number of bank products used
- `HasCrCard`: Credit card status (0 = No, 1 = Yes)
- `IsActiveMember`: Active membership status (0 = No, 1 = Yes)
- `EstimatedSalary`: Estimated salary of the customer
- `Exited`: Customer churn status (0 = No, 1 = Yes)

<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>PolynomialFeaturesFit function reference: <a href = 'https://docs.teradata.com/search/all?query=PolynomialFeaturesFit&value-filters=prodname~%2522Teradata+Package+for+Python%2522*vrm_release~%252220.00.00.03%2522&content-lang=en-US&_gl=1*19tr550*_gcl_aw*R0NMLjE3MzMyMDc4MjguRUFJYUlRb2JDaE1JeVpYM3BQNktpZ01WSWpLREF4MmluUmowRUFBWUFTQUFFZ0tSRVBEX0J3RQ..*_gcl_au*MTM2MDk0NzQ4OS4xNzM3NTI3NTA5*_ga*NTU2MTUwNDQ1LjE2OTM4MDU3NjE.*_ga_7PE2TMW3FE*MTczODY1MTA2OS4xNDUuMS4xNzM4NjUxMzgwLjYwLjAuMA..'>here</a></li>
    <li>PolynomialFeaturesTransform function reference: <a href = 'https://docs.teradata.com/search/all?query=PolynomialFeaturesTransform&value-filters=prodname~%2522Teradata+Package+for+Python%2522*vrm_release~%252220.00.00.03%2522&content-lang=en-US&_gl=1*19tr550*_gcl_aw*R0NMLjE3MzMyMDc4MjguRUFJYUlRb2JDaE1JeVpYM3BQNktpZ01WSWpLREF4MmluUmowRUFBWUFTQUFFZ0tSRVBEX0J3RQ..*_gcl_au*MTM2MDk0NzQ4OS4xNzM3NTI3NTA5*_ga*NTU2MTUwNDQ1LjE2OTM4MDU3NjE.*_ga_7PE2TMW3FE*MTczODY1MTA2OS4xNDUuMS4xNzM4NjUxMzgwLjYwLjAuMA..'>here</a></li></li>
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>